In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, Aer
from qiskit_algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit.circuit import ParameterVector

In [3]:
# Prepare graph data...
import os
from collections import namedtuple
Graph = namedtuple('Graph', ['X', 'Ri', 'Ro', 'y'])
class GraphDataset():
    def __init__(self, input_dir, n_samples=None):
        input_dir = os.path.expandvars(input_dir)
        filenames = [os.path.join(input_dir, f) for f in os.listdir(input_dir)
                     if f.startswith('event') and f.endswith('.npz')]
        self.filenames = (
            filenames[:n_samples] if n_samples is not None else filenames)

    def __getitem__(self, index):
        return load_graph(self.filenames[index])

    def __len__(self):
        return len(self.filenames)
def get_dataset(input_dir,n_files):
    return GraphDataset(input_dir, n_files)
def load_graph(filename):
    graph_dir = os.path.join(os.getcwd(), 'graphs')
    # Construct the full path to the specified file
    full_path = os.path.join(graph_dir, filename)
    """Read a single graph NPZ"""
    with np.load(full_path) as f:
        return sparse_to_graph(**dict(f.items()))
def sparse_to_graph(X, Ri_rows, Ri_cols, Ro_rows, Ro_cols, y, dtype=np.float32):
    n_nodes, n_edges = X.shape[0], Ri_rows.shape[0]
    Ri = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ro = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ri[Ri_rows, Ri_cols] = 1
    Ro[Ro_rows, Ro_cols] = 1
    return Graph(X, Ri, Ro, y)
#Function to load X, Ri, Ro, Y.
def load_raw(graph_name,xyr):
    graph_ex=load_graph(graph_name)
    #Load raw data
    y=graph_ex.y
    Ri=graph_ex.Ri
    Ro=graph_ex.Ro
    X=graph_ex.X
    if xyr=='X':
        return X
    elif xyr=='Ri':
        return Ri
    elif xyr=='Ro':
        return Ro
    else:
        return y

In [4]:
#Load Ri, Ro edge data of dimension Nv x Ne.
Ri=load_raw('event000001000_g000.npz','Ri')
Ro=load_raw('event000001000_g000.npz','Ro')
Ri,Ro=Ri.T,Ro.T
Rio=Ri+Ro

In [5]:
#Load X data of dimension Nv x 4(including color)
import os
import pandas as pd
#dir_ = os.path.join(os.getcwd(), 'color')
dir_ = os.path.join(os.getcwd(), 'coloredX')
file_path = os.path.join(dir_, f"{'event000001000_g000'}.csv")
v = pd.read_csv(file_path)
print("features data loaded...")

features data loaded...


In [6]:
#Convenient representation of edge data 
#edges = [[i1,j1], [i2,j2], ... ]; i1, i2,... are outgoing-nodes, and j1, j2, ... are incoming-nodes
import json
graph='event000001000_g000'
with open("./networks/"+graph+".json", "r") as json_file:
        _,_,edges= json.load(json_file)
print("edge data loaded...")

edge data loaded...


In [7]:
#Reduced adjacency matrix A
#A[i][j]==1 implies there is an edge between i, j-th nodes.
A=[[0 for _ in range(10)] for _ in range(10)]
red_edges=[]
for s,e in edges:
    if s<10 and e<10:
        red_edges.append([s,e])
for s,e in red_edges:
    A[s][e]=1
print("Adjacency matrix prepared...")

Adjacency matrix prepared...


In [8]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
# Reduced colored X data
raw_data=v[:10].values
# Create a quantum circuit with Nv qubits
Nv = len(raw_data)
num_qubits = Nv * 4  # 4 qubits per row (x,y,z,color)
# Initialize a quantum circuit
qc = QuantumCircuit(num_qubits)
feature = [Parameter(f'phi_{i}') for i in range(num_qubits)]
# Encode each feature value into a qubit 
for i in range(Nv):
    for j in range(4):
        qc.ry(feature[i * 4 + j], i * 4 + j)
    #Entangle 3 positions
    qc.cx(i * 4, i * 4 + 1)
    qc.cx(i * 4 + 1, i * 4 + 2)
    qc.cx(i * 4 + 2, i * 4)
    #Entangle color and position-x.
    qc.cx(i * 4, i * 4 + 3)
# Encode each feature value into a qubit 
print("All 4 features encoded in QC...")
# Define a list of trainable parameters for the entanglement
num_params = len(A)  # Number of trainable parameters for the entanglement
ansatz = [Parameter(f'theta_{i}') for i in range(num_params)]
print("Entangling qubits using fake adjacency matrix...")
# Apply trainable entanglement gates based on adjacency matrix A
for i in range(len(A)):
    for j in range(i + 1, len(A)):
        if A[i][j] == 1:
            # Use trainable parameters for entanglement angles
            qc.cx(i * 4, j * 4)
            qc.ry(ansatz[i], i * 4)
            qc.ry(ansatz[j], j * 4)
print("Parameterized quantum circuit ready...")

All 4 features encoded in QC...
Entangling qubits using fake adjacency matrix...
Parameterized quantum circuit ready...


In [87]:
# We need to convert the 4*Nv qubits to Ne qubits circuit. 
# To do so, measure relevant Ne qubits.
# Define the list of measurement circuits
measurement_circuits = []
print("Creating a measurement circuit...")
# Iterate over pairs of nodes (i, j)
for i in range(len(A)):
    for j in range(i + 1, len(A)):
        if A[i][j] == 1:
            # Create a new quantum circuit for measurement
            measure_circuit = qc.copy()
            print("Measured...")
            # Measure the qubits corresponding to nodes i and j
            measure_circuit.measure(i * 4, i * 4)
            measure_circuit.measure(j * 4, j * 4)
            # Add the measurement circuit to the list
            measurement_circuits.append(measure_circuit)

Creating a measurement circuit...
...


CircuitError: 'Index 0 out of range for size 0.'

In [56]:
# Prepare data for training (quantum circuits and labels)
X = measurement_circuits
y = load_raw('event000001000_g000.npz','y')
# For now, only look at first 10 edges.
Y = y[:10]  # True edge labels of dimension Ne=10
Ne = 10
# Create the Quantum Neural Network (QNN) classifier
qnn = NeuralNetworkBinaryClassifier(
    circuit = qc,optimizer=COBYLA(),
    quantum_instance = Aer.get_backend('qasm_simulator'),
    training_input_size = len(X),
    datapoints = len(X))

# Callback function to plot the loss
def callback_graph(epochs, loss, val_loss):
    plt.figure()
    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.legend()
    plt.show()

# Train the QNN with binary cross-entropy loss
qnn.fit(X, Y, callback=callback_graph)

Creating Q(G)NN...
